# Importing Packages and Inicial Data Preview

In [10]:
# Importing packages

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from pprint import pprint

from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.pipeline import Pipeline
from collections import Counter

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix, make_scorer, accuracy_score, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.decomposition import PCA

from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC

from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import make_pipeline

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier

import umap.umap_ as umap
from functools import partial

import optuna
from optuna.visualization import plot_pareto_front
import plotly

In [4]:
import sklearn
import optuna
import scikeras
import tensorflow as tf
import imblearn

print("scikit-learn:", sklearn.__version__)
print("optuna:", optuna.__version__)
print("scikeras:", scikeras.__version__)
print("tensorflow:", tf.__version__)
print("imblearn:", imblearn.__version__)

scikit-learn: 1.6.1
optuna: 4.3.0
scikeras: 0.13.0
tensorflow: 2.19.0
imblearn: 0.13.0


In [5]:
# loading dataset and handle a subset of it
path_2 = "datasets/diabetes_binary_health_indicators_BRFSS2015.csv"

df = pd.read_csv(path_2)

# Models Implementation

In [6]:
# Step 1: Split features and target
X = df.drop(columns=['Diabetes_binary'])
y = df['Diabetes_binary']
print("Before balancing:", Counter(y))

# Step 2: Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Step 3: Apply undersampling
undersample = RandomUnderSampler(sampling_strategy={0.0: 35000}, random_state=17)
X_bal, y_bal = undersample.fit_resample(X_train, y_train)
print("After undersampling:", Counter(y_bal))

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

pca = PCA(n_components=5) 

X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

Before balancing: Counter({0.0: 218334, 1.0: 35346})
After undersampling: Counter({0.0: 35000, 1.0: 28277})


## Testing Hyperparameter Tuning With Optuna Optimizations

### SVM Model

In [ ]:
# Defining tuning optimization for SVM

def objective_svc(trial):
    # Suggest hyperparameters to tune
    C = trial.suggest_loguniform("C", 1e-3, 100)
    gamma = trial.suggest_loguniform("gamma", 1e-4, 1e-1)
    n_components = trial.suggest_int("n_components", 200, 1000)  # optional tuning for Nystroem dimension

    # Build pipeline with these hyperparameters
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('nystroem', Nystroem(kernel='rbf', gamma=gamma, n_components=n_components, random_state=42)),
        ('clf', LinearSVC(C=C, class_weight='balanced', random_state=42, max_iter=10000)),
    ])

    
    scoring = {
        "accuracy": make_scorer(accuracy_score),
        "precision": make_scorer(precision_score, average='macro'),
        "recall": make_scorer(recall_score, average='macro'),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Cross-validate with multiple metrics (define 'scoring' dict before calling this)
    scores = cross_validate(pipeline, X_train, y_train, scoring=scoring, cv=cv, n_jobs=-1)
    
    # Return tuple of metrics for multi-objective optimization
    return scores["test_accuracy"].mean(), scores["test_precision"].mean(), scores["test_recall"].mean()


study_svc = optuna.create_study(directions=["maximize", "maximize", "maximize"], study_name="SVC")
study_svc.optimize(objective_svc, n_trials=30)

plot_pareto_front(study_svc, target_names=["Accuracy", "Precision", "Recall"]).show()

[I 2025-05-16 02:12:06,588] A new study created in memory with name: SVC
C:\Users\PC\AppData\Local\Temp\ipykernel_23812\1837004999.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 1e-3, 100)
C:\Users\PC\AppData\Local\Temp\ipykernel_23812\1837004999.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform("gamma", 1e-4, 1e-1)
[I 2025-05-16 02:15:40,987] Trial 0 finished with values: [0.7001882149107228, 0.6199978994232449, 0.7355297319278328] and parameters: {'C': 13.41805372370113, 'gamma': 0.005197743336103416, 'n_components': 852}.
C:\Users\PC\AppData\Local\Temp\ipykernel_23812\1837004999.py:5: FutureWar

### Best SVM Model

In [5]:
# Best parameters for each model

best_trial_svm = max(study_svc.best_trials, key=lambda t: sum(t.values))
print("SVC Best Params:", best_trial_svm.params)
print("Accuracy:", best_trial_svm.values[0])
print("Precision:", best_trial_svm.values[1])
print("Recall:", best_trial_svm.values[2])

est_trial = study_svc.best_trials[0] 

best_params = best_trial_svm.params
print("Best params:", best_params)

# Build pipeline with the best parameters
best_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('nystroem', Nystroem(kernel='rbf',
                         gamma=best_params["gamma"],
                         n_components=best_params["n_components"],
                         random_state=42)),
    ('clf', LinearSVC(C=best_params["C"],
                      class_weight='balanced',
                      random_state=42,
                      max_iter=10000))
])

# Fit the pipeline on full training data
best_pipeline.fit(X_train, y_train)

# Predict on test set
y_pred = best_pipeline.predict(X_test)

# Print classification report
print("Classification Report for Best SVC Model:")
print(classification_report(y_test, y_pred))

SVC Best Params: {'C': 0.0022349107112217994, 'gamma': 0.0001809469736933914, 'n_components': 353}
Accuracy: 0.7400169452918518
Precision: 0.6237037325723807
Recall: 0.7227155576261659
Best params: {'C': 0.0022349107112217994, 'gamma': 0.0001809469736933914, 'n_components': 353}


Classification Report for Best SVC Model:
              precision    recall  f1-score   support

         0.0       0.94      0.75      0.83     43667
         1.0       0.31      0.69      0.43      7069

    accuracy                           0.74     50736
   macro avg       0.62      0.72      0.63     50736
weighted avg       0.85      0.74      0.78     50736



### SVM Model

In [ ]:
# Defining tuning optimization for RF

def objective_rf(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 600),
        "max_depth": trial.suggest_int("max_depth", 5, 60),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "class_weight": "balanced",
        "n_jobs": -1
    }

    model = RandomForestClassifier(**params)

    scoring = {
        "accuracy": make_scorer(accuracy_score),
        "precision": make_scorer(precision_score, average='macro'),
        "recall": make_scorer(recall_score, average='macro'),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    scores = cross_validate(model, X_train, y_train, scoring=scoring, cv=cv)
    return scores["test_accuracy"].mean(), scores["test_precision"].mean(), scores["test_recall"].mean()

study_rf = optuna.create_study(directions=["maximize", "maximize", "maximize"], study_name="RF")
study_rf.optimize(objective_rf, n_trials=30)

plot_pareto_front(study_rf, target_names=["Accuracy", "Precision", "Recall"]).show()

[I 2025-05-16 02:26:19,040] A new study created in memory with name: RF
C:\Users\PC\AppData\Local\Temp\ipykernel_23812\4108633504.py:5: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.

c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\distributions.py:699: UserWarning:

The distribution is specified by [100, 300] and step=600, but the range is not divisible by `step`. It will be replaced by [100, 100].

C:\Users\PC\AppData\Local\Temp\ipykernel_23812\4108633504.py:6: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.

c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\distributions.py:699: UserWarning:

The distribution is specified by [5, 20] and step=60, but the range is not divisible by `step`. It will be replaced by [5, 5].

C:\Users\PC\AppData\Local\Temp\ipykernel_23812\4108633504.py

### Best RF Model

In [8]:
best_trial_rf = max(study_rf.best_trials, key=lambda t: sum(t.values))
print("SVC Best Params:", best_trial_rf.params)
print("Accuracy:", best_trial_rf.values[0])
print("Precision:", best_trial_rf.values[1])
print("Recall:", best_trial_rf.values[2])

best_trial_rf = study_rf.best_trials[0]

best_params_rf = best_trial_rf.params
print("Best RF params:", best_params_rf)

# Building the RF model with the best hyperparameters
best_rf = RandomForestClassifier(
    n_estimators=best_params_rf["n_estimators"],
    max_depth=best_params_rf["max_depth"],
    min_samples_split=best_params_rf["min_samples_split"],
    min_samples_leaf=best_params_rf["min_samples_leaf"],
    class_weight="balanced",
    n_jobs=-1,
    random_state=42
)

best_rf.fit(X_train, y_train)
y_pred_rf = best_rf.predict(X_test)

# Print classification report
print("Classification Report for Best RF Model:")
print(classification_report(y_test, y_pred_rf))

SVC Best Params: {'n_estimators': 100, 'max_depth': 5, 'min_samples_split': 8, 'min_samples_leaf': 1}
Accuracy: 0.6767827717348881
Precision: 0.613960007738758
Recall: 0.7294311269164213
Best RF params: {'n_estimators': 100, 'max_depth': 5, 'min_samples_split': 8, 'min_samples_leaf': 1}
Classification Report for Best RF Model:
              precision    recall  f1-score   support

         0.0       0.95      0.65      0.77     43667
         1.0       0.27      0.80      0.41      7069

    accuracy                           0.67     50736
   macro avg       0.61      0.73      0.59     50736
weighted avg       0.86      0.67      0.72     50736



### NN Model

In [7]:
# Defining tuning optimization for NN

X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

def create_model(input_dim, units1, dropout1, n_layers, units_i, dropout_i, lr):
    model = Sequential()
    model.add(Dense(units1, activation="relu", input_shape=(input_dim,)))
    model.add(Dropout(dropout1))
    
    for _ in range(n_layers):
        model.add(Dense(units_i, activation="relu"))
        model.add(Dropout(dropout_i))
    
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer=Adam(learning_rate=lr),
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

def objective_nn(trial):
    units1 = trial.suggest_int("units1", 64, 256)
    dropout1 = trial.suggest_float("dropout1", 0.2, 0.5)
    n_layers = 2
    units_i = trial.suggest_int("units_i", 32, 128)
    dropout_i = trial.suggest_float("dropout_i", 0.2, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

    model = create_model(X_train_final.shape[1], units1, dropout1, n_layers, units_i, dropout_i, lr)

    early_stop = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

    model.fit(
        X_train_final, y_train_final,
        validation_data=(X_val, y_val),
        epochs=20,
        batch_size=batch_size,
        callbacks=[early_stop],
        verbose=0
    )

    # Predict on validation set
    y_pred_probs = model.predict(X_val)
    y_pred = (y_pred_probs > 0.5).astype(int)

    val_accuracy = accuracy_score(y_val, y_pred)
    val_precision = precision_score(y_val, y_pred, average='macro')
    val_recall = recall_score(y_val, y_pred, average='macro')

    return val_accuracy, val_precision, val_recall

study_nn = optuna.create_study(directions=["maximize", "maximize", "maximize"], study_name="NN")
study_nn.optimize(objective_nn, n_trials=30, timeout=1800)

plot_pareto_front(study_nn, target_names=["Accuracy", "Precision", "Recall"]).show()

[I 2025-05-16 12:05:09,716] A new study created in memory with name: NN
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1s 764us/step


c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2025-05-16 12:05:27,763] Trial 0 finished with values: [0.8606765379782699, 0.43033826898913496, 0.5] and parameters: {'units1': 244, 'dropout1': 0.3629071781776245, 'units_i': 88, 'dropout_i': 0.4577229596298902, 'lr': 0.00520307117566572, 'batch_size': 128}.
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1s 772us/step


[I 2025-05-16 12:07:32,128] Trial 1 finished with values: [0.8633866318460667, 0.7360560907223695, 0.5249914592007958] and parameters: {'units1': 164, 'dropout1': 0.20728777624609063, 'units_i': 74, 'dropout_i': 0.3936184502775568, 'lr': 0.000889674078257198, 'batch_size': 32}.
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2025-05-16 12:30:37,056] Trial 27 finished with values: [0.8606765379782699, 0.43033826898913496, 0.5] and parameters: {'units1': 151, 'dropout1': 0.4438133607563678, 'units_i': 75, 'dropout_i': 0.49630014534168765, 'lr': 0.005305204648616968, 'batch_size': 128}.
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1s 768us/step


[I 2025-05-16 12:31:01,167] Trial 28 finished with values: [0.8619083988272684, 0.7566892759668911, 0.5087189347885873] and parameters: {'units1': 102, 'dropout1': 0.24821819224738612, 'units_i': 52, 'dropout_i': 0.391968817301507, 'lr': 0.0014483562879309174, 'batch_size': 128}.
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1s 737us/step


[I 2025-05-16 12:32:19,993] Trial 29 finished with values: [0.8611200078839094, 0.7784799606215194, 0.502628976698108] and parameters: {'units1': 130, 'dropout1': 0.4344128077735063, 'units_i': 38, 'dropout_i': 0.361623081212049, 'lr': 0.0009789351958430152, 'batch_size': 32}.


### Best NN Model

In [ ]:
# Retrieve the best trial based on the sum of multi-objective metrics
best_trial_nn = max(study_nn.best_trials, key=lambda t: sum(t.values))

print("Best NN Parameters:", best_trial_nn.params)
print("Accuracy:", best_trial_nn.values[0])
print("Precision:", best_trial_nn.values[1])
print("Recall:", best_trial_nn.values[2])

# Extract parameters
params = best_trial_nn.params
units1 = params["units1"]
dropout1 = params["dropout1"]
units_i = params["units_i"]
dropout_i = params["dropout_i"]
lr = params["lr"]
batch_size = params["batch_size"]
n_layers = 2  # fixed as per your tuning setup

# Create the best model
def create_model_for_best(input_dim, units1, dropout1, n_layers, units_i, dropout_i, learning_rate):
    model = Sequential()
    model.add(Dense(units1, activation="relu", input_shape=(input_dim,)))
    model.add(Dropout(dropout1))
    for _ in range(n_layers):
        model.add(Dense(units_i, activation="relu"))
        model.add(Dropout(dropout_i))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=["accuracy", "Precision", "Recall"]
    )
    return model

# Instantiate and train the best model
best_model = create_model_for_best(X_train.shape[1], units1, dropout1, n_layers, units_i, dropout_i, lr)

early_stop = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

best_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=batch_size,
    class_weight=class_weights,
    callbacks=[early_stop],
    verbose=0
)

y_pred_nn = best_model.predict(X_test)
y_pred_nn = (y_pred_nn > 0.5).astype(int)

print("Classification Report for Best NN Model:")
print(classification_report(y_test, y_pred_nn))


Best NN Parameters: {'units1': 86, 'dropout1': 0.343719727890171, 'units_i': 72, 'dropout_i': 0.31609477137387, 'lr': 0.0002279311582385667, 'batch_size': 128}
Accuracy: 0.8623518687329079
Precision: 0.7760067201750985
Recall: 0.5101622374732456


c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1586/1586 ━━━━━━━━━━━━━━━━━━━━ 1s 736us/step
Classification Report for Best NN Model:
              precision    recall  f1-score   support

         0.0       0.95      0.65      0.78     43667
         1.0       0.28      0.81      0.41      7069

    accuracy                           0.68     50736
   macro avg       0.62      0.73      0.59     50736
weighted avg       0.86      0.68      0.73     50736



# Best Model Obtained

In [ ]:
# Best model and the parameters tuned
print("The best model obtained (SVC) and it's params:", best_trial_svc.params)

cm = confusion_matrix(y_test, y_pred_model)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues")
plt.show()